In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
import os
import warnings
warnings.filterwarnings('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/dataset.csv')


Mounted at /content/drive


In [5]:
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [26]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

unique_ids = sorted(df['SystemCodeNumber'].unique())

df[["Timestamp", "SystemCodeNumber", "Occupancy", "Capacity"]].to_csv("df_model1.csv", index=False)
df[['ID','SystemCodeNumber', 'Capacity', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'Timestamp']].to_csv("df_model2.csv", index=False)
df[['ID','SystemCodeNumber', 'Capacity', 'Occupancy', 'Latitude', 'Longitude', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'Timestamp']].to_csv("df_model3.csv", index=False)

In [30]:
#input data streams for model 1
schemas1 = []
data_streams1 = []

for i, lot_id in enumerate(unique_ids, start=1):
    lot_df = df[df['SystemCodeNumber'] == lot_id].copy()
    lot_df = lot_df[['Timestamp', 'SystemCodeNumber', 'Occupancy', 'Capacity']]

    fname = f"df_model1_lot{i}.csv"
    lot_df.to_csv(fname, index=False)

    schema = pw.schema_from_csv(fname)
    data = pw.demo.replay_csv(fname, schema=schema, input_rate=9000)

    schemas1.append(schema)
    data_streams1.append(data)

#data_streams1[i] for i-th parking lot for 1st Model

#-------------------------------------------------------------------------------

#input data streams for model 2
schemas2 = []
data_streams2 = []

for i, lot_id in enumerate(unique_ids, start=1):
    lot_df = df[df['SystemCodeNumber'] == lot_id].copy()
    lot_df = lot_df[['ID','SystemCodeNumber', 'Capacity', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'Timestamp']]

    fname = f"df_model2_lot{i}.csv"
    lot_df.to_csv(fname, index=False)

    schema = pw.schema_from_csv(fname)
    data = pw.demo.replay_csv(fname, schema=schema, input_rate=9000)

    schemas2.append(schema)
    data_streams2.append(data)

#data_streams2[i] for i-th parking lot for 2nd Model

#-------------------------------------------------------------------------------

#input data streams for model 3
schemas3 = []
data_streams3 = []

for i, lot_id in enumerate(unique_ids, start=1):
    lot_df = df[df['SystemCodeNumber'] == lot_id].copy()
    lot_df = lot_df[['ID','SystemCodeNumber', 'Capacity', 'Occupancy', 'Latitude', 'Longitude', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'Timestamp']]

    fname = f"df_model3_lot{i}.csv"
    lot_df.to_csv(fname, index=False)

    schema = pw.schema_from_csv(fname)
    data = pw.demo.replay_csv(fname, schema=schema, input_rate=9000)

    schemas3.append(schema)
    data_streams3.append(data)

#data_streams3[i] for i-th parking lot for 3rd Model

In [46]:
for i, stream in enumerate(data_streams1, start=1):
    print(f"\n===== data_streams1[{i-1}] (Lot {i}) =====")
    display(stream)


===== data_streams1[0] (Lot 1) =====


<pathway.Table schema={'Timestamp': <class 'str'>, 'SystemCodeNumber': <class 'str'>, 'Occupancy': <class 'int'>, 'Capacity': <class 'int'>}>


===== data_streams1[1] (Lot 2) =====


<pathway.Table schema={'Timestamp': <class 'str'>, 'SystemCodeNumber': <class 'str'>, 'Occupancy': <class 'int'>, 'Capacity': <class 'int'>}>


===== data_streams1[2] (Lot 3) =====


...

<pathway.Table schema={'Timestamp': <class 'str'>, 'SystemCodeNumber': <class 'str'>, 'Occupancy': <class 'int'>, 'Capacity': <class 'int'>}>


===== data_streams1[12] (Lot 13) =====


<pathway.Table schema={'Timestamp': <class 'str'>, 'SystemCodeNumber': <class 'str'>, 'Occupancy': <class 'int'>, 'Capacity': <class 'int'>}>


===== data_streams1[13] (Lot 14) =====


<pathway.Table schema={'Timestamp': <class 'str'>, 'SystemCodeNumber': <class 'str'>, 'Occupancy': <class 'int'>, 'Capacity': <class 'int'>}>

In [48]:
for i, stream in enumerate(data_streams2, start=1):
    print(f"\n===== data_streams1[{i-1}] (Lot {i}) =====")
    display(stream)


===== data_streams1[0] (Lot 1) =====


<pathway.Table schema={'ID': <class 'int'>, 'SystemCodeNumber': <class 'str'>, 'Capacity': <class 'int'>, 'Occupancy': <class 'int'>, 'VehicleType': <class 'str'>, 'TrafficConditionNearby': <class 'str'>, 'QueueLength': <class 'int'>, 'IsSpecialDay': <class 'int'>, 'Timestamp': <class 'str'>}>


===== data_streams1[1] (Lot 2) =====


<pathway.Table schema={'ID': <class 'int'>, 'SystemCodeNumber': <class 'str'>, 'Capacity': <class 'int'>, 'Occupancy': <class 'int'>, 'VehicleType': <class 'str'>, 'TrafficConditionNearby': <class 'str'>, 'QueueLength': <class 'int'>, 'IsSpecialDay': <class 'int'>, 'Timestamp': <class 'str'>}>


===== data_streams1[2] (Lot 3) =====


...

<pathway.Table schema={'ID': <class 'int'>, 'SystemCodeNumber': <class 'str'>, 'Capacity': <class 'int'>, 'Occupancy': <class 'int'>, 'VehicleType': <class 'str'>, 'TrafficConditionNearby': <class 'str'>, 'QueueLength': <class 'int'>, 'IsSpecialDay': <class 'int'>, 'Timestamp': <class 'str'>}>


===== data_streams1[12] (Lot 13) =====


<pathway.Table schema={'ID': <class 'int'>, 'SystemCodeNumber': <class 'str'>, 'Capacity': <class 'int'>, 'Occupancy': <class 'int'>, 'VehicleType': <class 'str'>, 'TrafficConditionNearby': <class 'str'>, 'QueueLength': <class 'int'>, 'IsSpecialDay': <class 'int'>, 'Timestamp': <class 'str'>}>


===== data_streams1[13] (Lot 14) =====


<pathway.Table schema={'ID': <class 'int'>, 'SystemCodeNumber': <class 'str'>, 'Capacity': <class 'int'>, 'Occupancy': <class 'int'>, 'VehicleType': <class 'str'>, 'TrafficConditionNearby': <class 'str'>, 'QueueLength': <class 'int'>, 'IsSpecialDay': <class 'int'>, 'Timestamp': <class 'str'>}>

In [54]:
for i, stream in enumerate(data_streams3, start=1):
    print(f"\n===== data_streams1[{i-1}] (Lot {i}) =====")
    display(stream)


===== data_streams1[0] (Lot 1) =====


<pathway.Table schema={'ID': <class 'int'>, 'SystemCodeNumber': <class 'str'>, 'Capacity': <class 'int'>, 'Occupancy': <class 'int'>, 'Latitude': <class 'float'>, 'Longitude': <class 'float'>, 'VehicleType': <class 'str'>, 'TrafficConditionNearby': <class 'str'>, 'QueueLength': <class 'int'>, 'IsSpecialDay': <class 'int'>, 'Timestamp': <class 'str'>}>


===== data_streams1[1] (Lot 2) =====


<pathway.Table schema={'ID': <class 'int'>, 'SystemCodeNumber': <class 'str'>, 'Capacity': <class 'int'>, 'Occupancy': <class 'int'>, 'Latitude': <class 'float'>, 'Longitude': <class 'float'>, 'VehicleType': <class 'str'>, 'TrafficConditionNearby': <class 'str'>, 'QueueLength': <class 'int'>, 'IsSpecialDay': <class 'int'>, 'Timestamp': <class 'str'>}>


===== data_streams1[2] (Lot 3) =====


...

<pathway.Table schema={'ID': <class 'int'>, 'SystemCodeNumber': <class 'str'>, 'Capacity': <class 'int'>, 'Occupancy': <class 'int'>, 'Latitude': <class 'float'>, 'Longitude': <class 'float'>, 'VehicleType': <class 'str'>, 'TrafficConditionNearby': <class 'str'>, 'QueueLength': <class 'int'>, 'IsSpecialDay': <class 'int'>, 'Timestamp': <class 'str'>}>


===== data_streams1[12] (Lot 13) =====


<pathway.Table schema={'ID': <class 'int'>, 'SystemCodeNumber': <class 'str'>, 'Capacity': <class 'int'>, 'Occupancy': <class 'int'>, 'Latitude': <class 'float'>, 'Longitude': <class 'float'>, 'VehicleType': <class 'str'>, 'TrafficConditionNearby': <class 'str'>, 'QueueLength': <class 'int'>, 'IsSpecialDay': <class 'int'>, 'Timestamp': <class 'str'>}>


===== data_streams1[13] (Lot 14) =====


<pathway.Table schema={'ID': <class 'int'>, 'SystemCodeNumber': <class 'str'>, 'Capacity': <class 'int'>, 'Occupancy': <class 'int'>, 'Latitude': <class 'float'>, 'Longitude': <class 'float'>, 'VehicleType': <class 'str'>, 'TrafficConditionNearby': <class 'str'>, 'QueueLength': <class 'int'>, 'IsSpecialDay': <class 'int'>, 'Timestamp': <class 'str'>}>

**Model 1: Baseline Linear Model**

In [50]:
import datetime
from bokeh.palettes import Category10
import bokeh.plotting
import panel as pn
pn.extension()

fmt = "%Y-%m-%d %H:%M:%S"

# Store all lot plots
all_viz_panels1 = []

for i, data in enumerate(data_streams1):
    data_with_time = data.with_columns(
        t = data.Timestamp.dt.strptime(fmt),
        day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
    )

    delta_window = (
        data_with_time.windowby(
            pw.this.t,
            instance=pw.this.day,
            window=pw.temporal.tumbling(datetime.timedelta(days=1)),
            behavior=pw.temporal.exactly_once_behavior()
        )
        .reduce(
            t=pw.this._pw_window_end,
            occ_max=pw.reducers.max(pw.this.Occupancy),
            occ_min=pw.reducers.min(pw.this.Occupancy),
            cap=pw.reducers.max(pw.this.Capacity)
        )
        .with_columns(
            price = 10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
        )
    )

    def make_plot(source, lot_index=i+1):
        fig = bokeh.plotting.figure(
            height=400,
            width=800,
            title=f"Model 1: Lot {lot_index}",
            x_axis_type="datetime"
        )
        fig.line("t", "price", source=source, line_width=2, color="navy")
        fig.circle("t", "price", source=source, size=5, color="red")
        return fig

    viz = delta_window.plot(make_plot, sorting_col="t")
    all_viz_panels1.append((f"Lot {i+1}", pn.panel(viz)))  # for tabs

# ✅ One single servable panel at the end
pn.Tabs(*all_viz_panels1).servable()


Tabs
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [3] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [4] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [5] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [6] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [7] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [8] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [9] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [10] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [11] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [12] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [13] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [47]:
%%capture --no-display
pw.run()

Output()

**Model 2: Demand-Based Price Function**

In [51]:
import datetime
import pathway as pw
import panel as pn
import bokeh.plotting
from bokeh.palettes import Category10

pn.extension()

fmt = "%Y-%m-%d %H:%M:%S"
alpha, beta, gamma, delta, epsilon = 0.4, 0.1, 0.15, 0.1, 0.05
base_price, lambda_val = 10.0, 0.7

all_viz_panels2 = []

for i, data in enumerate(data_streams2):
    data_with_time2 = data.with_columns(
        t = data.Timestamp.dt.strptime(fmt=fmt),
        day = data.Timestamp.dt.strptime(fmt=fmt).dt.strftime("%Y-%m-%dT00:00:00"),
    )

    data_casted = data_with_time2.with_columns(
        occupancy_f = pw.cast(float, pw.this.Occupancy),
        capacity_f = pw.cast(float, pw.this.Capacity),
        traffic_f = pw.apply_with_type(
            lambda x: 0.0 if str(x).lower() == "low" else 1.0 if str(x).lower() == "average" else 2.0,
            float,
            pw.this.TrafficConditionNearby
        ),
        queue_f = pw.cast(float, pw.this.QueueLength),
        special_f = pw.cast(float, pw.this.IsSpecialDay),
        vehicle_weight = pw.if_else(
            pw.this.VehicleType == "car", 1.0,
            pw.if_else(pw.this.VehicleType == "bike", 0.5,
                       pw.if_else(pw.this.VehicleType == "truck", 1.5, 1.0))
        )
    )

    data_demand = data_casted.with_columns(
        demand = (
            alpha * (pw.this.occupancy_f / pw.this.capacity_f) +
            beta * pw.this.queue_f +
            gamma * pw.this.traffic_f +
            delta * pw.this.special_f +
            epsilon * pw.this.vehicle_weight
        )
    )

    window2 = (
        data_demand.windowby(
            pw.this.t,
            instance=pw.this.day,
            window=pw.temporal.tumbling(datetime.timedelta(days=1)),
            behavior=pw.temporal.exactly_once_behavior()
        )
        .reduce(
            t=pw.this._pw_window_end,
            avg_demand = pw.reducers.avg(pw.this.demand)
        )
        .with_columns(
            normalized_demand = pw.this.avg_demand / 2.0
        )
        .with_columns(
            price = base_price * (1 + lambda_val * pw.this.normalized_demand)
        )
        .with_columns(
            price_bounded = pw.apply(lambda x: max(5.0, min(20.0, x)), pw.this.price)
        )
    )

    def plot_model2(source, lot=i+1):
        fig = bokeh.plotting.figure(
            height=400,
            width=900,
            title=f"Model 2: Lot {lot}",
            x_axis_type="datetime"
        )
        fig.line("t", "price_bounded", source=source, line_width=2, color="navy")
        fig.circle("t", "price_bounded", source=source, size=5, color="red")
        return fig

    viz = window2.plot(plot_model2, sorting_col="t")
    all_viz_panels2.append((f"Lot {i+1}", pn.panel(viz)))

# ✅ One single servable panel at the end
pn.Tabs(*all_viz_panels2).servable()

Tabs
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [3] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [4] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [5] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [6] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [7] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [8] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [9] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [10] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [11] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [12] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [13] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [52]:
%%capture --no-display
pw.run()

Output()

**Model 3: Competitive Pricing Model**

In [53]:
import pathway as pw
import datetime
import panel as pn
import bokeh.plotting

# # Step 1: Load CSV
# fname = "df_model3.csv"
# schema = pw.schema_from_csv(fname)
# data3 = pw.demo.replay_csv(fname, schema=schema, input_rate=9000)

# Step 2: Parse timestamps
fmt = "%Y-%m-%d %H:%M:%S"

all_viz_panels3 = []

alpha, beta, gamma, delta, epsilon = 0.4, 0.1, 0.15, 0.1, 0.05
base_price, lambda_val = 10.0, 0.7

for i, data in enumerate(data_streams3):
    data_with_time3 = data.with_columns(
        t = data.Timestamp.dt.strptime(fmt=fmt),
        day = data.Timestamp.dt.strptime(fmt=fmt).dt.strftime("%Y-%m-%dT00:00:00"),
    )

    data_casted = data_with_time3.with_columns(
        occupancy_f = pw.cast(float, pw.this.Occupancy),
        capacity_f = pw.cast(float, pw.this.Capacity),
        latitude_f = pw.cast(float, pw.this.Latitude),
        longitude_f = pw.cast(float, pw.this.Longitude),
        traffic_f = pw.apply_with_type(
            lambda x: 0.0 if str(x).lower() == "low"
            else 1.0 if str(x).lower() == "average"
            else 2.0,
            float,
            pw.this.TrafficConditionNearby
        ),
        queue_f = pw.cast(float, pw.this.QueueLength),
        special_f = pw.cast(float, pw.this.IsSpecialDay),
        vehicle_weight = pw.if_else(
            pw.this.VehicleType == "car", 1.0,
            pw.if_else(pw.this.VehicleType == "bike", 0.5,
                       pw.if_else(pw.this.VehicleType == "truck", 1.5, 1.0))
        )
    )

    data_demand = data_casted.with_columns(
        demand = (
            alpha * (pw.this.occupancy_f / pw.this.capacity_f) +
            beta * pw.this.queue_f +
            gamma * pw.this.traffic_f +
            delta * pw.this.special_f +
            epsilon * pw.this.vehicle_weight
        )
    )

    window3 = (
        data_demand.windowby(
            pw.this.t,
            instance=pw.this.day,
            window=pw.temporal.tumbling(datetime.timedelta(days=1)),
            behavior=pw.temporal.exactly_once_behavior()
        )
        .reduce(
            t=pw.this._pw_window_end,
            avg_demand = pw.reducers.avg(pw.this.demand),
            min_demand = pw.reducers.min(pw.this.demand),
            max_demand = pw.reducers.max(pw.this.demand),
            avg_latitude = pw.reducers.avg(pw.this.latitude_f),
            avg_longitude = pw.reducers.avg(pw.this.longitude_f),
            lot_count = pw.reducers.count()
        )
        .with_columns(
            normalized_demand = pw.if_else(
                pw.this.max_demand == pw.this.min_demand,
                0.0,
                (pw.this.avg_demand - pw.this.min_demand) / (pw.this.max_demand - pw.this.min_demand)
            )
        )
        .with_columns(
            competition_factor = pw.apply_with_type(
                lambda count: max(0.0, min(1.0, 1.0 - (count - 1) / 10.0)),
                float, pw.this.lot_count
            ),
            location_premium = pw.apply_with_type(
                lambda lat, lon: 0.1 if (lat > 26.1 and lon > 91.7) else 0.0,
                float, pw.this.avg_latitude, pw.this.avg_longitude
            )
        )
        .with_columns(
            price = base_price * (
                1 + lambda_val * pw.this.normalized_demand +
                0.2 * pw.this.location_premium -
                0.2 * (1 - pw.this.competition_factor)
            )
        )
        .with_columns(
            price_bounded = pw.apply(lambda x: max(5.0, min(25.0, x)), pw.this.price)
        )
    )

    def plot_model3(source, lot=i+1):
        fig = bokeh.plotting.figure(
            height=400,
            width=900,
            title=f"Model 3: Lot {lot}",
            x_axis_type="datetime"
        )
        fig.line("t", "price_bounded", source=source, line_width=2, color="navy")
        fig.circle("t", "price_bounded", source=source, size=5, color="red")
        return fig

    viz = window3.plot(plot_model3, sorting_col="t")
    all_viz_panels3.append((f"Lot {i+1}", pn.panel(viz)))

# ✅ One single servable panel at the end
pn.Tabs(*all_viz_panels3).servable()


Tabs
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [3] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [4] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [5] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [6] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [7] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [8] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [9] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [10] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [11] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [12] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [13] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [55]:
%%capture --no-display
pw.run()

Output()